In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy
import sys
import time

In [2]:
class Graph:
    
    def __init__(self, n):
        self.n_nodes = n
        self.has_matrix = False
        self.matrix_error_size = None
        try:
            self.matrix = np.zeros((n, n), dtype="uint8")
            self.has_matrix = True
        except MemoryError as error:
            print("Cannot create matrix >>", error)
            ea = str(error)
            eb = ea[ea.index("iB for")-1]
            eb_m = 10**3 if eb=="G" else 10**6
            self.matrix_error_size = float(ea[ea.index("allocate")+9:ea.index("iB for")-2]) * eb_m
        self.nodes = [set() for i in range(n)]
    
    
    def add_edge(self, v, w):
        
        if v == w:
            if self.has_matrix:
                self.matrix[v-1, v-1] = 2
            self.nodes[v-1].add(v)
            
        else:
            if self.has_matrix:
                self.matrix[v-1, w-1] = 1
                self.matrix[w-1, v-1] = 1
            self.nodes[v-1].add(w)
            self.nodes[w-1].add(v)
    
    def get_node(self, v):
        return self.nodes[v-1]
    
    def get_lists(self):
        return self.nodes
    
    def get_node_edges(self):
        return {i+1:self.nodes[i] for i in range(self.n_nodes)}
    
    def get_matrix(self):
        if self.has_matrix:
            return self.matrix
        return None
    
    def get_matrix_beautiful(self):
        if self.has_matrix:
            return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
        return None
    
    def sort_neighbors(self):
        self.nodes = [sorted(n) for n in self.nodes]

In [3]:
def open_graph_txt(filename, extra=False):
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        n_nodes = int(lines[0])
        edges = [tuple(map(lambda i: int(i), line.split(" "))) for line in lines[1:-1]]
    
    graph = Graph(n_nodes)
    for v, w in edges:
        graph.add_edge(v, w)
    
    if extra:
        return graph, n_nodes, edges
    
    return graph

In [4]:
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    if graph.has_matrix:
        print("Número de arestas:", graph.get_matrix().sum()/2)
        print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
        print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
      

      

In [9]:
class MinimumPath:
    
    def __init__(self, graph):
        self.graph = graph
        self.matrix = np.full((graph.n_nodes, graph.n_nodes), fill_value=-1, dtype="int32")
        self.run()
    
    def run(self):
        for v in tqdm(range(1, self.graph.n_nodes+1)):
            bfsl = BFSL(self.graph, v)
            bfsl.search()
            for bfsl_node_index in np.argwhere(bfsl.visited == 1).reshape(-1):
                self.matrix[v-1, bfsl_node_index] = bfsl.level[bfsl_node_index]
            del bfsl
    
    def get_distance(self, u, v):
        return self.matrix[u-1, v-1]
    
    def get_diameter(self):
        return np.max(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.graph.n_nodes+1), index=np.arange(1, self.graph.n_nodes+1))
    

In [10]:
class Components:
    
    def __init__(self, graph):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.components = []
        
        while np.argwhere(self.visited == 0).reshape(-1).shape[0] > 0:
            root = np.argwhere(self.visited == 0).reshape(-1)[0] + 1

            bfsl = BFSL(self.graph, root)
            bfsl.search()
            
            bfsl_visited_index = np.argwhere(bfsl.visited == 1).reshape(-1)
            
            self.visited[bfsl_visited_index] = 1
            self.components.append((bfsl_visited_index+1).tolist())

    def get_components(self):
        a = sorted(self.components, key=lambda x: len(x), reverse=True)
        b = [len(x) for x in a]
        c = list(zip(b, a))
        return c


In [ ]:
dfsm = DFSM(graph, 1)
dfsm.search()

print("DFSM Result:")
dfsm_df = pd.DataFrame(list(zip(range(1, dfsm.graph.n_nodes+1), dfsm.level, dfsm.parent)), columns=["node", "level", "parent"], index=np.arange(1, dfsm.graph.n_nodes+1))
dfsm_df.to_csv("./outputs/dfsm_test.csv")

In [ ]:
minpath = MinimumPath(graph)

print("Minpath Distance:", minpath.get_distance(1, 3))
print("Minpath Diameter:", minpath.get_diameter())

In [ ]:
components = Components(graph)

with open("./outputs/components_test.txt", mode="w") as out:
    for line in components.get_components():
        out.write(str(line))
        out.write("\n")